In [7]:
from typing import Iterable
from sampo.schemas import Time
from sampo.scheduler.multi_agency.block_generator import generate_blocks, SyntheticBlockGraphType
from sampo.scheduler.multi_agency.multi_agency import StochasticManager, ScheduledBlock
from sampo.scheduler.multi_agency import Agent, validate_block_schedule
from uuid import uuid4
from copy import deepcopy

from sampo.generator import SimpleSynthetic
from sampo.scheduler import HEFTScheduler
from random import Random

r_seed = 231
rand = Random(r_seed)
ss = SimpleSynthetic(rand)

scheduler_constructors = [HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler]
base_contractor = ss.contractor(10)

bg = generate_blocks(SyntheticBlockGraphType.SEQUENTIAL, 20, [1, 1, 1], lambda x: (50, 60), 0.5, rand)

def finish_time(schedule: Iterable[ScheduledBlock]) -> Time:
    return max([sblock.end_time for sblock in schedule])

SEQUENTIAL SEQUENTIAL SEQUENTIAL PARALLEL GENERAL SEQUENTIAL SEQUENTIAL GENERAL GENERAL SEQUENTIAL PARALLEL PARALLEL SEQUENTIAL SEQUENTIAL GENERAL SEQUENTIAL PARALLEL GENERAL SEQUENTIAL PARALLEL GENERAL


In [8]:
from sampo.schemas import WorkerProductivityMode, Contractor, IntervalGaussian
from sampo.schemas.time_estimator import DefaultWorkEstimator, WorkTimeEstimator
from sampo.scheduler.multi_agency import Manager
from typing import Callable


def construct_work_estimator(i: int, productivity: WorkerProductivityMode) -> WorkTimeEstimator:
    work_estimator = DefaultWorkEstimator()
    work_estimator.set_productivity_mode(productivity)
    for worker in ['driver', 'fitter', 'manager', 'handyman', 'electrician', 'engineer']:
        work_estimator.set_worker_productivity(IntervalGaussian(0.2 * i + 0.2, 1, 0, 2), worker)
    return work_estimator


def construct_agent(i: int, scheduler_constructor, contractor: Contractor, productivity: WorkerProductivityMode) -> Agent:
    work_estimator = construct_work_estimator(i, productivity)
    
    return Agent(f'Agent {i}', scheduler_constructor(work_estimator=work_estimator), [contractor])

def test_with_manager(manager_constructor: Callable[[list[Agent]], Manager], productivity: WorkerProductivityMode):
    contractors = [deepcopy(base_contractor) for _ in scheduler_constructors]
    for contractor in contractors:
        contractor.id = str(uuid4())
    
    agents = [construct_agent(i, scheduler_constructors[i % len(scheduler_constructors)], contractor, productivity)
              for i, contractor in enumerate(contractors)]
    manager = manager_constructor(agents)
    
    scheduled_blocks = manager.manage_blocks(bg)
    # validate_block_schedule(bg, scheduled_blocks, agents)
    
    return finish_time(scheduled_blocks.values())

In [39]:
time_static = test_with_manager(Manager, WorkerProductivityMode.Stochastic)
time_static

9633

In [40]:
time_stochastic = test_with_manager(StochasticManager, WorkerProductivityMode.Stochastic)
time_stochastic

4531

In [41]:
def run_comparison(iterations: int):
    successes = 0
    for i in range(iterations):
        time_static = test_with_manager(Manager, WorkerProductivityMode.Stochastic)
        time_stochastic = test_with_manager(StochasticManager, WorkerProductivityMode.Stochastic)
        print(f'{time_static} {time_stochastic}')
        if time_stochastic < time_static:
            successes += 1
    print(f'Success percentage: {int(100 * successes / iterations)}% on {iterations} iterations')

run_comparison(10)

27254 11829
16765 51175
19170 43856
28078 18893
15790 13870
29057 6302
12200 8015
18533 15938
5346 8329
11771 12090
Success percentage: 60% on 10 iterations


# Mode 2

In [15]:
test_with_manager(Manager, WorkerProductivityMode.Stochastic)

2540

In [9]:
def test_in_mode(mode: WorkerProductivityMode):
    work_estimator = DefaultWorkEstimator()
    work_estimator.set_productivity_mode(mode)
    
    contractors = [deepcopy(base_contractor) for _ in scheduler_constructors]
    for contractor in contractors:
        contractor.id = str(uuid4())
    
    for worker in ['driver', 'fitter', 'manager', 'handyman', 'electrician', 'engineer']:
        for i, contractor in enumerate(contractors):
            work_estimator.set_worker_productivity(IntervalGaussian(0.2 * i + 0.2, 1, 0, 2), worker, contractor.id)
    
    agents = [Agent(f'Agent {i}', scheduler_constructors[i % len(scheduler_constructors)](work_estimator=work_estimator), [contractor])
              for i, contractor in enumerate(contractors)]
    manager = StochasticManager(agents)
    
    scheduled_blocks = manager.manage_blocks(bg)
    # validate_block_schedule(bg, scheduled_blocks, agents)
    
    ma_time = finish_time(scheduled_blocks.values())
    
    conjuncted = bg.to_work_graph()
    print(f'Size: {conjuncted.vertex_count}')
    scheduler = HEFTScheduler(work_estimator=work_estimator)
    conjuncted_time = scheduler.schedule(conjuncted, [contractors[-1]]).execution_time
    return ma_time, conjuncted_time

In [4]:
# test_in_mode(WorkerProductivityMode.Static)

In [10]:
test_in_mode(WorkerProductivityMode.Stochastic)

Size: 1506


(3662, 3610)

In [11]:
for _ in range(10):
    print(test_in_mode(WorkerProductivityMode.Stochastic))

Size: 1506
(3893, 3856)
Size: 1506
(3681, 3776)
Size: 1506
(3842, 3779)
Size: 1506
(3608, 3671)
Size: 1506
(3704, 3579)
Size: 1506
(3731, 3892)
Size: 1506
(3807, 3784)
Size: 1506
(3667, 3487)
Size: 1506
(3539, 3841)
Size: 1506
(3926, 3600)
